In [1]:
import json
import preprocessor as p
import pandas as pd
import numpy as np
import spacy
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk import stem
import string
import warnings
warnings.filterwarnings('ignore')
nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

('parser', <spacy.pipeline.pipes.DependencyParser at 0x2126683c4c8>)

In [2]:
## load json into dataframe
with open('tweets.json') as f:
    data=[]
    for line in f:
        if line!="\n":
            data.append(json.loads(line))

In [3]:
d = pd.DataFrame(data)
df = d[['id','created_at','text','lang']]
#df['hashtags']= np.array([i['entities']['hashtags']for i in data])
#df['hashtags']= [i['entities']['hashtags']for i in data]
#df

In [4]:
df

,id,created_at,text,lang
0,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,@FinisimaPersona 👍 https://t.co/Erego0NAgb,und
1,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @randallbell: 'better take my shirt off for...,en
2,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @rosapantin1301: De China sale un virus que...,es
3,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @LasVegasLocally: A @WetRepublic employee h...,en
4,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @maxvayshia: Nothing survives here. Not dre...,en
...,...,...,...,...
112795,1.238691e+18,Sat Mar 14 04:57:44 +0000 2020,RT @ilovehye: 유럽에 와보세요. 마스크 쓴 사람 한명도 없습니다. 왜냐구...,ko
112796,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @thestandardth: ผลโพลเผยความเห็นต่อการจัดกา...,th
112797,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @ggreenwald: One day we’ll find out how man...,en
112798,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @tristanmom2: Regular surgical masks does n...,en


In [5]:
#in case entities is not existing
hashtagspre=[]
for i in data:
    if 'entities'in i.keys():
        hashtagspre.append(i['entities']['hashtags'])
    else:
        hashtagspre.append('')
df['hashtags'] = hashtagspre
df

,id,created_at,text,lang,hashtags
0,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,@FinisimaPersona 👍 https://t.co/Erego0NAgb,und,[]
1,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @randallbell: 'better take my shirt off for...,en,[]
2,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @rosapantin1301: De China sale un virus que...,es,[]
3,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @LasVegasLocally: A @WetRepublic employee h...,en,[]
4,1.238599e+18,Fri Mar 13 22:51:34 +0000 2020,RT @maxvayshia: Nothing survives here. Not dre...,en,[]
...,...,...,...,...,...
112795,1.238691e+18,Sat Mar 14 04:57:44 +0000 2020,RT @ilovehye: 유럽에 와보세요. 마스크 쓴 사람 한명도 없습니다. 왜냐구...,ko,[]
112796,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @thestandardth: ผลโพลเผยความเห็นต่อการจัดกา...,th,"[{'text': 'หน้ากากอนามัย', 'indices': [53, 67]}]"
112797,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @ggreenwald: One day we’ll find out how man...,en,[]
112798,1.238691e+18,Sat Mar 14 04:57:45 +0000 2020,RT @tristanmom2: Regular surgical masks does n...,en,[]


# text in hashtags

In [6]:
## extract hashtags
hashtags_list = list(df.hashtags)
all_list=[]
for item in hashtags_list:
    text_list = []
    for i in item:
        text_list.append(i['text'].lower())
    all_list.append(text_list)
df['textlist'] = all_list

## extract english text 
df = df[df['lang']=='en']

# select emoji

In [7]:
# select emojis
def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)
emoji_list=[]
for i in df['text']:
    emoji_list.append(extract_emojis(str(i)))
df['emoji'] = emoji_list

# preprocessor data

In [8]:
# preprocessor clean data
clean_text=[]
for i in df['text']:
    clean_text.append(p.clean(str(i)))
df['clean_text'] = clean_text

In [9]:
# clean data function
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    #looping through conditions
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in string.punctuation:
            filtered_tweet.append(w)
#    return filtered_tweet
    return ' '.join(filtered_tweet)

In [10]:
# calling clean data 
filter_text=[]
for i in df['clean_text']:
    filter_text.append(clean_tweets(str(i)))
df['filter_text'] = filter_text

In [11]:
# blob
#from textblob import TextBlob
#blob = TextBlob(filtered_tweet)

# Normalization and Lemmatization

In [12]:
#@Tokenize
def spacy_tokenize(string):
    tokens = list()
    doc = nlp(string)
    for token in doc:
        tokens.append(token)
    return tokens

In [13]:
#@Normalize
def normalize(tokens):
    normalized_tokens = list()
    for token in tokens:
        normalized = token.text.lower().strip()
        if ((token.is_alpha or token.is_digit)):
            normalized_tokens.append(normalized)
    return normalized_tokens

In [14]:
#@Tokenize and normalize
def tokenize_normalize(string):
    return normalize(spacy_tokenize(string))

In [15]:
# normalize data
norm_text=[]
for i in df['filter_text']:
    norm_text.append(tokenize_normalize(str(i)))
df['norm_text'] = norm_text

In [16]:
#a
wordnet_lemm = stem.WordNetLemmatizer()
lemword =[]
for i in norm_text:
    lem_word = []
    for j in i:
        lem_word.append(wordnet_lemm.lemmatize(j))
    lemword.append(lem_word)
df['lem_word'] = lemword
# turn lem list into string
lem_word_list=[]
for i in df['lem_word']:
    lem_word_list.append(",".join(i))
df['lem'] = lem_word_list

# Drop duplicates

In [17]:
#drop duplicates
ddf = df.drop_duplicates(['lem'])
ddf.reset_index(inplace=True,drop=True)
dataframe = ddf.drop(['lem_word'],axis=1)

# create class for labeling

## text for labelling

In [18]:
# create six classes 
excitement=['pleased','amused','excited','excitement','exciting','astonished']
happy=['glad','delighted','satisfied','happy','joy','love','beabetteryou']
pleasant= ['content','at ease','relaxed','serene','pleasant']
surprise=[ 'sad','frustration','alarmed','afraid','tense','miserable','positive','RIP']
fear=['disgust','depression','fear','terrified','horrible','anxiety','stress','epidemic','worry']
angry=['distressed','gloomy','angry','annoyed','offended','discrimination','depressed','anxious','anger','mask']
labels =[excitement,happy,pleasant,surprise,fear,angry]

In [19]:
# input text data for class count
countlist = [[],[],[],[],[],[]]
for line in ddf.textlist:
    for i in range(0,len(countlist)):
        count = 0
        for tag in line:
            if tag in labels[i]:
                count += 1
        countlist[i].append(count)
# write the classlist into dataframe        
dataframe['excitement']=countlist[0]
dataframe['happy']=countlist[1]
dataframe['pleasant']=countlist[2]
dataframe['surprise']=countlist[3]
dataframe['fear']=countlist[4]
dataframe['angry']=countlist[5]



## emoji for labelling

In [20]:
# create emoji label
excitement_emoji=['😀','🤣','😃','😄','😗','😙','😚','☺️','🙂','🤗','🤩','🏆']
happy_emoji=['😆','😉','😊','😋','😎','😍','😘','🥰','🧡', '💛', '💚', '💙', '💜', '🔥']
pleasant_emoji= [ '😛','😜','😝','🤭','👏','👍','💘', '💝', '💟']
surprise_emoji=[ '😭','😦','😧','😨','😩','🤯','💔','☹️','🙁','😖','😞','😟','🙀']
fear_emoji=['😰','😱','🥵','🥶','🆘','😢','😭','🥵','🥶','💩']
angry_emoji=['🤢','🤮','🤧','👎','😤','😡','😠','🤬','🤮']
labels_emoji =[excitement_emoji,happy_emoji,pleasant_emoji,surprise_emoji,fear_emoji,angry_emoji]

In [21]:
# input emoji for class count
countlist_emoji = [[],[],[],[],[],[]]
for line_emoji in dataframe.emoji:
    for i in range(0,len(countlist_emoji)):
        count = 0
        for emojis in line_emoji:
            if emojis in labels_emoji[i]:
                count += 1
        countlist_emoji[i].append(count)

In [22]:
# write the classlist_emoji into dataframe 
dataframe['excitement_emoji']=countlist_emoji[0]
dataframe['happy_emoji']=countlist_emoji[1]
dataframe['pleasant_emoji']=countlist_emoji[2]
dataframe['surprise_emoji']=countlist_emoji[3]
dataframe['fear_emoji']=countlist_emoji[4]
dataframe['angry_emoji']=countlist_emoji[5]

In [23]:
# calculate the class score both in text and emoji
dataframe['excitementclass']=dataframe['excitement']+dataframe['excitement_emoji']
dataframe['happyclass']=dataframe['happy']+dataframe['happy_emoji']
dataframe['pleasantclass']=dataframe['pleasant']+dataframe['pleasant_emoji']
dataframe['surpriseclass']=dataframe['surprise']+dataframe['surprise_emoji']
dataframe['fearclass']=dataframe['fear']+dataframe['fear_emoji']
dataframe['angryclass']=dataframe['angry']+dataframe['angry_emoji']

# score  and labelling

In [24]:
# extract the numberic columnm to collect the max index for labeling classes
maxdataframe = dataframe[['excitementclass','happyclass','pleasantclass','surpriseclass','fearclass','angryclass']]
maxdataframe['max_value'] = maxdataframe.max(axis=1)
maxdataframe['max_idx'] = maxdataframe.idxmax(axis=1)

In [25]:
#maxdataframe

In [26]:
# write the max column in dataframe
dataframe['max_value'] = maxdataframe['max_value']
dataframe['max_idx'] = maxdataframe['max_idx']

In [27]:
#dataframe

# Sort label

In [28]:
import pandas as pd  
a = dataframe.drop_duplicates(subset=['max_idx'],keep='first')
data_st = dataframe.loc[dataframe['max_idx'].isin(a['max_idx'])]
data_st = data_st.sort_values('max_idx',ascending=True) 
#ata_cf = frame.loc[frame['pop'].isin(a['pop'])]

In [29]:
#data_st

In [30]:
excitement = data_st.loc[data_st["max_idx"] == "excitementclass"]

# write to csv

In [31]:
#excitement = data_st.loc[data_st["max_idx"] == "excitementclass"].head()
excitement.to_csv('excitement.csv')

In [32]:
happy = data_st.loc[data_st["max_idx"] == "happyclass"]
happy.to_csv('happy.csv')

In [33]:
pleasant = data_st.loc[data_st["max_idx"] == "pleasantclass"]
pleasant.to_csv('pleasant.csv')

In [34]:
surprise = data_st.loc[data_st["max_idx"] == "surpriseclass"]
surprise.to_csv('surprise.csv')

In [35]:
fear = data_st.loc[data_st["max_idx"] == "fearclass"]
fear.to_csv('fear.csv')

In [37]:
angry = data_st.loc[data_st["max_idx"] == "angryclass"]
angry.to_csv('angry.csv')